In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model

In [2]:
nelec = pd.read_csv('nelec_082323.csv')
dhw = pd.read_csv('dhw_merge.csv')

### Domain Classification

In [3]:
nelec = nelec.drop(['YEAR'], axis=1)
dhw = dhw.drop(['YEAR'], axis=1)

In [4]:
input_nelec = nelec.iloc[:-1,].reset_index(drop=True)
output_nelec = nelec[['n_elec']].iloc[1:].reset_index(drop=True)
output_nelec.columns = ['nelec']

input_dhw = dhw.iloc[:-1,].reset_index(drop=True)
output_dhw = dhw[['DHW']].iloc[1:].reset_index(drop=True)
output_dhw.columns = ['dhw']

nelec2 = pd.concat([input_nelec, output_nelec], axis=1)
dhw2 = pd.concat([input_dhw, output_dhw], axis=1)

In [5]:
scaler_nelec = MinMaxScaler()
scaler_dhw = MinMaxScaler()

scaler_nelec.fit(nelec)
scaler_dhw.fit(dhw)

scaled_nelec = scaler_nelec.transform(nelec)
scaled_dhw = scaler_dhw.transform(dhw)

nelec = pd.DataFrame(scaled_nelec, index=nelec.index, columns=nelec.columns)
dhw = pd.DataFrame(scaled_dhw, index=dhw.index, columns=dhw.columns)

In [6]:
standard_nelec = int(len(nelec) * 0.7)
standard_dhw = int(len(dhw) * 0.7)

nelec_train = nelec.iloc[:standard_nelec]
nelec_test = nelec.iloc[standard_nelec:].reset_index(drop=True)

dhw_train = dhw.iloc[:standard_dhw]
dhw_test = dhw.iloc[standard_dhw:].reset_index(drop=True)

In [7]:
nelec_train.shape

(20417, 12)

In [8]:
nelec_test.shape

(8751, 12)

In [9]:
nelec_trainx = nelec_train.drop(['n_elec'], axis=1)
nelec_trainy = nelec_train[['n_elec']]

nelec_testx = nelec_test.drop(['n_elec'], axis=1)
nelec_testy = nelec_test[['n_elec']]

dhw_trainx = dhw_train.drop(['DHW'], axis=1)
dhw_trainy = dhw_train[['DHW']]

dhw_testx = dhw_test.drop(['DHW'], axis=1)
dhw_testy = dhw_test[['DHW']]

In [10]:
seqLength = 24
def buildDataSet(input, target, seqLength):
    xdata = []
    ydata = []
    for i in range(len(input) - seqLength):
        tx = input.iloc[i:i+seqLength]
        ty = target.iloc[i+seqLength-1]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)

nelec_trainx, nelec_trainy = buildDataSet(nelec_trainx, nelec_trainy, seqLength)
nelec_testx, nelec_testy = buildDataSet(nelec_testx, nelec_testy, seqLength)

dhw_trainx, dhw_trainy = buildDataSet(dhw_trainx, dhw_trainy, seqLength)
dhw_testx, dhw_testy = buildDataSet(dhw_testx, dhw_testy, seqLength)

In [11]:
print(nelec_trainx.shape)
print(nelec_trainy.shape)
print(nelec_testx.shape)
print(nelec_testy.shape)
print(dhw_trainx.shape)
print(dhw_trainy.shape)
print(dhw_testx.shape)
print(dhw_testy.shape)

(20393, 24, 11)
(20393, 1)
(8727, 24, 11)
(8727, 1)
(20393, 24, 11)
(20393, 1)
(8727, 24, 11)
(8727, 1)


In [12]:
# Convert NumPy arrays to TensorFlow tensors
nelec_trainx_tensor = tf.constant(nelec_trainx, dtype=tf.float32)
nelec_trainy_tensor = tf.constant(nelec_trainy, dtype=tf.float32)

nelec_testx_tensor = tf.constant(nelec_testx, dtype=tf.float32)
nelec_testy_tensor = tf.constant(nelec_testy, dtype=tf.float32)

dhw_trainx_tensor = tf.constant(dhw_trainx, dtype=tf.float32)
dhw_trainy_tensor = tf.constant(dhw_trainy, dtype=tf.float32)

dhw_testx_tensor = tf.constant(dhw_testx, dtype=tf.float32)
dhw_testy_tensor = tf.constant(dhw_testy, dtype=tf.float32)

# Create TensorFlow datasets
nelec_train = tf.data.Dataset.from_tensor_slices((nelec_trainx_tensor, nelec_trainy_tensor))
nelec_test = tf.data.Dataset.from_tensor_slices((nelec_testx_tensor, nelec_testy_tensor))

dhw_train = tf.data.Dataset.from_tensor_slices((dhw_trainx_tensor, dhw_trainy_tensor))
dhw_test = tf.data.Dataset.from_tensor_slices((dhw_testx_tensor, dhw_testy_tensor))

# Define batch sizes
batch_size_nelec_train = 512
batch_size_nelec_test = 512
batch_size_dhw_train = 512
batch_size_dhw_test = 512

# Create data loaders
NELEC_train = nelec_train.batch(batch_size_nelec_train).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
NELEC_test = nelec_test.batch(batch_size_nelec_test).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
DHW_train = dhw_train.batch(batch_size_dhw_train).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
DHW_test = dhw_test.batch(batch_size_dhw_test).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# 모델 구조 선언

### GradientReversalLayer

In [13]:
class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GradientReversalLayer, self).__init__()

    def call(self, x):
        return tf.negative(x), x

### LSTM for Feature Extraction

In [35]:
class lstm(tf.keras.models.Model):
    def __init__(self):
        super(lstm, self).__init__()
        self.lstm1 = tf.keras.layers.LSTM(64, return_sequences=True, return_state=True)
        self.bn = tf.keras.layers.BatchNormalization()
        self.lstm2 = tf.keras.layers.LSTM(64, return_sequences=True)

        self.dropout = tf.keras.layers.Dropout(0.6)
        self.relu = tf.keras.layers.ReLU()

    def call(self, x):
        x, h0, c0 = self.lstm1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.lstm2(x, initial_state=[h0, c0])
        x = self.bn(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = x[:,-1,:]
        return x

### Domain Regression

In [36]:
class domain_regression(tf.keras.Model):
    def __init__(self):
        super(domain_regression, self).__init__()
        self.regression_layer1 = tf.keras.layers.Dense(100)
        self.regression_layer4 = tf.keras.layers.Dense(1)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(0.6)
        self.relu = tf.keras.layers.ReLU()

    def call(self, x):
        x = self.bn1(self.regression_layer1(x))
        x = self.relu(x)
        x = self.dropout(x)
        x = self.regression_layer4(x)
        return x

### Domain Classification

In [37]:
class domain_classfication(tf.keras.Model):
    def __init__(self):
        super(domain_classfication, self).__init__()
        self.classification_layer0 = GradientReversalLayer()
        self.classification_layer1 = tf.keras.layers.Dense(100)
        self.classification_layer4 = tf.keras.layers.Dense(1)
        self.relu = tf.keras.layers.ReLU()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(0.6)

    def call(self, x):
        x = self.classification_layer0(x)
        x = self.classification_layer1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.classification_layer4(x)
        return x

### Set the DANN

In [38]:
class DANN(tf.keras.Model):
    def __init__(self, lstm):
        super(DANN, self).__init__()
        self.lstm = lstm
        self.regression = domain_regression()
        self.classification = domain_classfication()

    def call(self, x):
        feature = self.lstm(x)
        reg_output = self.regression(feature)
        cla_output = self.classification(feature)

        return reg_output, cla_output

### Loss 선언

In [39]:
class DANNLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=1.0):
        super(DANNLoss, self).__init__()
        self.alpha = alpha
        self.reg_loss = tf.keras.losses.MeanAbsoluteError()
        self.cla_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    def call(self, y_true, y_pred, domain_num):
        reg_output, cla_output = y_pred
        batch_size = tf.shape(reg_output)[0]
        cla_target = tf.fill((batch_size, 1), tf.cast(domain_num, dtype=tf.float32))

        reg_loss = self.reg_loss(y_true[1], reg_output)
        cla_loss = self.cla_loss(cla_target, cla_output)
        cla_loss2 = cla_loss * self.alpha

        loss = reg_loss + cla_loss2

        return loss

In [40]:
# 모델 생성
my_lstm = lstm()
model = DANN(my_lstm)
# 손실 함수 생성
alpha_value = 1.0  # 원하는 alpha 값으로 설정
loss_fn = DANNLoss(alpha=alpha_value)

# 훈련

In [42]:
# Define hyperparameters
alpha = 1.0
epochs = 200
patience = 100
counter = 10

# Define the learning rate schedule
initial_learning_rate = 0.001
end_learning_rate = 0.0
total_steps = epochs * len(NELEC_train)

# Define the optimizer
optimizer = tf.keras.optimizers.Adamax(learning_rate=0.001, clipnorm=0.01)

# Initialize variables for early stopping
best_loss = float('inf')
best_model_weights = None

# Training loop
for i in range(1, epochs + 1):
    total_loss = 0
    reg_loss_total = 0
    cla_loss_total = 0

    for step, (source_data, target_data) in enumerate(zip(NELEC_train, DHW_train)):
        source_x = source_data[0].numpy()  # Convert to NumPy array
        source_y = source_data[1].numpy()
        target_x = target_data[0].numpy()
        target_y = target_data[1].numpy()

        # Forward pass
        source_result = model(source_x)
        target_result = model(target_x)

        # Calculate losses
        source_loss, source_reg_loss, source_cla_loss = loss_fn(source_result, source_y, 0, alpha=alpha)  # Source domain label 0
        target_loss, target_reg_loss, target_cla_loss = loss_fn(target_result, target_y, 1, alpha=alpha)  # Target domain label 1

        loss = source_loss + target_loss
        reg_loss = source_reg_loss + target_reg_loss
        cla_loss = source_cla_loss + target_cla_loss

        reg_loss_total += reg_loss
        cla_loss_total += cla_loss
        total_loss += loss

        # Compute gradients and update weights
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Early stopping check
    if reg_loss_total / len(NELEC_train) < best_loss:
        best_loss = reg_loss_total / len(NELEC_train)
        best_model_weights = model.get_weights()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping at epoch {i}')
            break

    print(f'Epoch : {i}, Total Avg Loss : {total_loss / len(NELEC_train):.4f}')
    print(f'Avg Regression Loss : {reg_loss_total / len(NELEC_train):.4f}')
    print(f'Avg Classification Loss : {cla_loss_total / len(NELEC_train):.4f}\n')

# Save the best model
if best_model_weights is not None:
    model.set_weights(best_model_weights)
    model.save('091823_in_the_lab.h5')

ValueError: Exception encountered when calling layer "domain_classfication_3" (type domain_classfication).

Layer "dense_14" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor: shape=(512, 64), dtype=float32, numpy=
array([[-0.08159236, -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [-0.07610296, -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [-0.07097369, -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       ...,
       [-0.03693607, -0.        , -0.00799986, ..., -0.00270401,
        -0.        , -0.        ],
       [-0.03417342, -0.        , -0.00669707, ..., -0.00293911,
        -0.        , -0.        ],
       [-0.03219561, -0.        , -0.00472287, ..., -0.00241153,
        -0.        , -0.        ]], dtype=float32)>, <tf.Tensor: shape=(512, 64), dtype=float32, numpy=
array([[0.08159236, 0.        , 0.        , ..., 0.        , 0.        ,

# train 데이터로 테스트

In [ ]:
# 저장된 최상의 소스 모델 불러오기
best_source_model2 = dann(my_lstm).to(device)
best_source_model2.load_state_dict(torch.load('091823_in_the_lab.pth'))
best_source_model2.eval()  # 평가 모드로 설정

s_pred = []  # MNIST 데이터셋의 예측값을 저장할 리스트
t_pred = []  # SVHN 데이터셋의 예측값을 저장할 리스트
s_real = []
t_real = []
s_d_pred = []
t_d_pred = []

for step, (source_data, target_data) in enumerate(zip(NELEC_train, DHW_train)):
    sourcex = source_data[0].to(device)
    sourcey = source_data[1].to(device)
    targetx = target_data[0].to(device)
    targety = target_data[1].to(device)

    # 저장된 최상의 모델로 예측
    source_domain_label_pred, source_pred = best_source_model2(sourcex)
    target_domain_label_pred, target_pred = best_source_model2(targetx)

    ## 예측값을 리스트에 저장
    s_pred.extend(source_pred.detach().cpu().numpy())
    t_pred.extend(target_pred.detach().cpu().numpy())
    s_real.extend(sourcey.detach().cpu().numpy())
    t_real.extend(targety.detach().cpu().numpy())
    s_d_pred.extend(source_domain_label_pred.detach().cpu().numpy())
    t_d_pred.extend(target_domain_label_pred.detach().cpu().numpy())

# R2 스코어 계산
s_r2 = r2_score(s_real, s_pred)
t_r2 = r2_score(t_real, t_pred)

print('SOURCE R2 Score:', s_r2)
print('TARGET R2 Score:', t_r2)

In [ ]:
plt.figure(figsize=(40,10))
plt.plot(s_real, color='red', label='source real')
plt.plot(s_pred, color='blue', label='source pred')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(40,10))
plt.plot(t_real, color='red', label='target real')
plt.plot(t_pred, color='blue', label='target pred')
plt.legend()
plt.show()

# test 데이터로 테스트

In [ ]:
# 저장된 최상의 소스 모델 불러오기
best_source_model2 = dann(my_lstm).to(device)
best_source_model2.load_state_dict(torch.load('091823_in_the_lab.pth'))
best_source_model2.eval()  # 평가 모드로 설정

s_pred = []  # MNIST 데이터셋의 예측값을 저장할 리스트
t_pred = []  # SVHN 데이터셋의 예측값을 저장할 리스트
s_real = []
t_real = []


for step, (source_data, target_data) in enumerate(zip(NELEC_test, DHW_test)):
    sourcex = source_data[0].to(device)
    sourcey = source_data[1].to(device)
    targetx = target_data[0].to(device)
    targety = target_data[1].to(device)

    # 저장된 최상의 모델로 예측
    _, source_pred = best_source_model2(sourcex)
    _, target_pred = best_source_model2(targetx)

    ## 예측값을 리스트에 저장
    s_pred.extend(source_pred.detach().cpu().numpy())
    t_pred.extend(target_pred.detach().cpu().numpy())
    s_real.extend(sourcey.detach().cpu().numpy())
    t_real.extend(targety.detach().cpu().numpy())

# R2 스코어 계산
s_r2 = r2_score(s_real, s_pred)
t_r2 = r2_score(t_real, t_pred)

print('SOURCE R2 Score:', s_r2)
print('TARGET R2 Score:', t_r2)

In [ ]:
plt.figure(figsize=(40,10))
plt.plot(s_real, color='red', label='source real')
plt.plot(s_pred, color='blue', label='source pred')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(40,10))
plt.plot(t_real, color='red', label='target real')
plt.plot(t_pred, color='blue', label='target pred')
plt.legend()
plt.show()

In [ ]:
import winsound as sd
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 800     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()

### Domain Embedding Space 시각화

In [ ]:
# 가장 앞 batch의 250개씩의 데이터만 샘플링
source_tsne = DataLoader(nelec_train,
                        batch_size=1000,
                        shuffle=False,  
                        drop_last=True)

target_tsne = DataLoader(dhw_train,
                        batch_size=1000,
                        shuffle=False,  
                        drop_last=True)

source_tsne2 = next(iter(source_tsne))
target_tsne2 = next(iter(target_tsne))

source_x_TNSE = source_tsne2[0].to(device)
source_y_TNSE = source_tsne2[1].to(device)

target_x_TNSE = target_tsne2[0].to(device)
target_y_TNSE = target_tsne2[1].to(device)

# 학습된 모델의 LSTM 부분만 활용 (100차원 임베딩 벡터를 받아오는 과정)
source_vector = model.lstm(source_x_TNSE)
target_vector = model.lstm(target_x_TNSE)

In [ ]:
from sklearn.manifold import TSNE

df = pd.DataFrame(np.concatenate([source_vector.cpu().detach().numpy(), target_vector.cpu().detach().numpy()]))

tsne_np = TSNE(n_components=2).fit_transform(df)
tsne_df = pd.DataFrame(tsne_np, columns=['component 0', 'component 1'])

In [ ]:
tsne_df_0 = tsne_df.loc[:1000]
tsne_df_1 = tsne_df.loc[1000:]

plt.scatter(tsne_df_0['component 0'], tsne_df_0['component 1'], color='green', label='NELEC', alpha=0.5)
plt.scatter(tsne_df_1['component 0'], tsne_df_1['component 1'], color='black', label='DHW', alpha=0.5)

plt.title('alpha = '+ str(alpha))
plt.xlabel('component 0')
plt.ylabel('component 1')
plt.legend()
plt.show()